Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [9]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from sklearn.linear_model import LogisticRegression
from sklearn import metrics


First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---


Part 1a: Logistic regression with regularization.

In [28]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)


def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  # Do not reshape the labels as we are using simple logistic regression
  #labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

trainData = np.reshape(train_dataset, (train_dataset.shape[0],image_size*image_size))
testData = np.reshape(test_dataset, (test_dataset.shape[0],image_size*image_size))
validData = np.reshape(valid_dataset, (valid_dataset.shape[0],image_size*image_size))

print('Training data shape: ', trainData.shape)
print('Testing data shape: ', testData.shape)


# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
model = model.fit(trainData[:500], train_labels[:500])

# check the accuracy on the validation set
print('Accuracy on validation set:',100*model.score(validData, valid_labels))

# check the accuracy on the testing set
print('Accuracy on test set:', 100*model.score(testData, test_labels))


print('Applying regularization to the model')
# instantiate a logistic regression model, and fit with X and y, regularization C = 0.5 --> lower the value higher the regularization
model = LogisticRegression(C=0.5)
model = model.fit(trainData[:500], train_labels[:500])

# check the accuracy on the validation set
print('Accuracy on validation set:',100*model.score(validData, valid_labels))

# check the accuracy on the testing set
print('Accuracy on test set:', 100*model.score(testData, test_labels))


Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)
Training data shape:  (200000, 784)
Testing data shape:  (10000, 784)
Accuracy on validation set: 75.36
Accuracy on test set: 82.21
Applying regularization to the model
Accuracy on validation set: 75.83
Accuracy on test set: 82.49


Part 1b: Neural network with regularization.

In [38]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

hid1_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hid1_nodes]))
  biases1 = tf.Variable(tf.zeros([hid1_nodes]))

  weights2 = tf.Variable(
    tf.truncated_normal([hid1_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    

  logits2 = tf.matmul(logits1, weights2) + biases2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)+0.01*(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(biases1)+tf.nn.l2_loss(weights2)+tf.nn.l2_loss(biases2)))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2)+biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),weights2)+biases2)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)
Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [39]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  #tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
    
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 3492.460693
Training accuracy: 9.0%
Validation accuracy: 39.6%
Loss at step 100: 1149.662231
Training accuracy: 90.3%
Validation accuracy: 79.0%
Loss at step 200: 421.240845
Training accuracy: 99.7%
Validation accuracy: 80.0%
Loss at step 300: 154.677475
Training accuracy: 99.8%
Validation accuracy: 80.9%
Loss at step 400: 56.996849
Training accuracy: 99.4%
Validation accuracy: 83.3%
Loss at step 500: 21.274799
Training accuracy: 96.6%
Validation accuracy: 85.2%
Loss at step 600: 8.210360
Training accuracy: 93.6%
Validation accuracy: 85.5%
Loss at step 700: 3.426486
Training accuracy: 91.6%
Validation accuracy: 85.5%
Loss at step 800: 1.671366
Training accuracy: 90.6%
Validation accuracy: 85.6%
Test accuracy: 90.8%


This model performs better compared to the NN without regularization which overfits the training data as below.
Training accuracy: 99.9%
Validation accuracy: 79.7%
Test accuracy: 86.0


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Part 2a: Use a very small batch of data compared to the above model.

In [31]:
# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
model = model.fit(trainData[:100], train_labels[:100])

# check the accuracy on the validation set
print('Accuracy on validation set:',100*model.score(validData, valid_labels))

# check the accuracy on the testing set
print('Accuracy on test set:', 100*model.score(testData, test_labels))


print('Applying regularization to the model')
# instantiate a logistic regression model, and fit with X and y, regularization C = 0.5 --> lower the value higher the regularization
model = LogisticRegression(C=0.5)
model = model.fit(trainData[:100], train_labels[:100])

# check the accuracy on the validation set
print('Accuracy on validation set:',100*model.score(validData, valid_labels))

# check the accuracy on the testing set
print('Accuracy on test set:', 100*model.score(testData, test_labels))


Accuracy on validation set: 67.18
Accuracy on test set: 73.27
Applying regularization to the model
Accuracy on validation set: 67.33
Accuracy on test set: 73.08


Part 2b: Use a very small batch of data to demonstrate the overfitting and use of regularization.

In [50]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 500

hid1_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hid1_nodes]))
  biases1 = tf.Variable(tf.zeros([hid1_nodes]))

  weights2 = tf.Variable(
    tf.truncated_normal([hid1_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    

  logits2 = tf.matmul(logits1, weights2) + biases2
  #loss = tf.reduce_mean(
    #tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)+0.05*(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(biases1)+tf.nn.l2_loss(weights2)+tf.nn.l2_loss(biases2)))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2)+biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),weights2)+biases2)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)
Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [51]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  #tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
    
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16132.662109
Training accuracy: 9.4%
Validation accuracy: 40.2%
Loss at step 100: 99.779388
Training accuracy: 100.0%
Validation accuracy: 76.7%
Loss at step 200: 1.469176
Training accuracy: 95.6%
Validation accuracy: 78.0%
Loss at step 300: 0.826881
Training accuracy: 95.6%
Validation accuracy: 78.0%
Loss at step 400: 0.814687
Training accuracy: 95.8%
Validation accuracy: 78.0%
Loss at step 500: 0.810895
Training accuracy: 95.8%
Validation accuracy: 78.0%
Loss at step 600: 0.808783
Training accuracy: 95.8%
Validation accuracy: 78.0%
Loss at step 700: 0.807254
Training accuracy: 95.8%
Validation accuracy: 78.1%
Loss at step 800: 0.806077
Training accuracy: 95.8%
Validation accuracy: 78.0%
Test accuracy: 84.0%


As seen above, especially in cases with small data sets the test accuracy is better compared to without regularization (as below):
Training accuracy: 100.0%
Validation accuracy: 76.1%
Test accuracy: 81.9%

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

Part 3a: NN with dropout

In [56]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

hid1_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hid1_nodes]))
  biases1 = tf.Variable(tf.zeros([hid1_nodes]))

  weights2 = tf.Variable(
    tf.truncated_normal([hid1_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits1 = tf.nn.relu(tf.matmul(tf_train_dataset, tf.nn.dropout(weights1, 0.5)) + biases1)
    

  logits2 = tf.matmul(logits1, weights2) + biases2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
  #loss = tf.reduce_mean(
  #  tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)+0.01*(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(biases1)+tf.nn.l2_loss(weights2)+tf.nn.l2_loss(biases2)))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2)+biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),weights2)+biases2)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)
Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [57]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  #tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
    
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 448.221497
Training accuracy: 12.9%
Validation accuracy: 35.3%
Loss at step 100: 118.108826
Training accuracy: 66.1%
Validation accuracy: 76.1%
Loss at step 200: 28.348370
Training accuracy: 79.0%
Validation accuracy: 80.9%
Loss at step 300: 22.498501
Training accuracy: 78.1%
Validation accuracy: 81.2%
Loss at step 400: 20.424015
Training accuracy: 82.4%
Validation accuracy: 81.9%
Loss at step 500: 23.153498
Training accuracy: 79.9%
Validation accuracy: 77.4%
Loss at step 600: 41.006924
Training accuracy: 77.3%
Validation accuracy: 82.1%
Loss at step 700: 20.076534
Training accuracy: 80.7%
Validation accuracy: 79.0%
Loss at step 800: 12.266539
Training accuracy: 83.7%
Validation accuracy: 82.0%
Test accuracy: 87.9%


This model performs better compared to the NN without regularization which overfits the training data as below. Training accuracy: 99.9% Validation accuracy: 79.7% Test accuracy: 86.0

Part 3b: NN with small batch (for overfitting case) and with dropout.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [139]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

image_size = 28
num_labels = 10


def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100000

hid1_nodes = 2048
hid2_nodes = 500
batch_size = 500

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  '''
  graph = tf.Graph()
  with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  '''
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hid1_nodes]))
  biases1 = tf.Variable(tf.zeros([hid1_nodes]))

  weights2 = tf.Variable(
    tf.truncated_normal([hid1_nodes, hid2_nodes]))
  biases2 = tf.Variable(tf.zeros([hid2_nodes]))

  weights3 = tf.Variable(
    tf.truncated_normal([hid2_nodes, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1)+ biases1)
  logits2 = tf.nn.relu(tf.matmul(logits1, weights2) + biases2)
  logits3 = tf.matmul(logits2, weights3) + biases3

  #loss = tf.reduce_mean(
  #  tf.nn.softmax_cross_entropy_with_logits(logits3, tf_train_labels))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits3, tf_train_labels)+0.005*(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(biases1)+tf.nn.l2_loss(weights2)+tf.nn.l2_loss(biases2)+tf.nn.l2_loss(weights3)+tf.nn.l2_loss(biases3)))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  #optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss) 
  
  
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.0 5, global_step, 100000, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits3)

  h1v = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  h2v = tf.nn.relu(tf.matmul(h1v, weights2) + biases2)
  valid_prediction = tf.nn.softmax(tf.matmul(h2v,weights3)+biases3)
  h1t = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  h2t = tf.nn.relu(tf.matmul(h1t, weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(h2t,weights3)+biases3)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)
Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [140]:
num_steps = 30001
batch_size = 500
with tf.Session(graph=graph) as session:
  #tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 12682.932617
Minibatch accuracy: 6.8%
Validation accuracy: 18.4%
Test accuracy: 19.2%
Minibatch loss at step 500: 7223.772949
Minibatch accuracy: 21.4%
Validation accuracy: 23.3%
Test accuracy: 25.7%
Minibatch loss at step 1000: 5626.082031
Minibatch accuracy: 29.2%
Validation accuracy: 29.1%
Test accuracy: 32.1%
Minibatch loss at step 1500: 4382.485840
Minibatch accuracy: 35.6%
Validation accuracy: 35.0%
Test accuracy: 38.7%
Minibatch loss at step 2000: 3413.770020
Minibatch accuracy: 38.0%
Validation accuracy: 40.9%
Test accuracy: 45.6%
Minibatch loss at step 2500: 2659.215576
Minibatch accuracy: 48.4%
Validation accuracy: 46.2%
Test accuracy: 51.7%
Minibatch loss at step 3000: 2071.625732
Minibatch accuracy: 53.0%
Validation accuracy: 51.2%
Test accuracy: 57.0%
Minibatch loss at step 3500: 1614.172974
Minibatch accuracy: 54.8%
Validation accuracy: 56.9%
Test accuracy: 62.6%
Minibatch loss at step 4000: 1257.637451
Minibatch accuracy: 65.6%
Valid

Dataset size: 100,000
hidden layer 1 = 2048
hidden layer 2 = 500
batch_size = 500

L2 regularization with beta=0.005


Minibatch loss at step 30000: 0.536853
Minibatch accuracy: 89.8%
Validation accuracy: 89.0%
Test accuracy: 93.6%